In [2]:
import pandas as pd
import re
import os
import numpy as np

In [3]:
from calcIoU import box_iou

a = [1, 1, 6, 4]
b = [ 2, 3, 4, 5]
lu = np.maximum(a[:2], b[:2])
rd = np.minimum(a[2:], b[2:])
print(lu)
print(rd)
print(box_iou(a, b))

[2 3]
[4 4]
0.117647058824


In [5]:
probs = {}
idlist = []
pattern = re.compile(r'\d{4}')
truths = {}
path = '/home/dj/data/CarPlane/label2'

with open("/home/dj/data/CarPlane/test.txt") as f:
    for line in f:
        #print (line)
        num = re.findall(pattern, line)
        num  = num[0]
        #print (num)
        idlist.append(num)
        
        if (num not in truths):
            truths[num] = []
        name = 'P' + num + '.txt'
        with open(path + '/' + name) as f5:
            for line in f5:
                line = line.strip()
                #print('line', line)
                linelist = line.split('\t')
                
                #print('linelist', linelist)
                truth = linelist[9: 13]
                #print('truth', truth)
                for index, item in enumerate(truth):
                    truth[index] = float(item)
                #print('truth2', truth)
                xmin, ymin, w, h = truth[0], truth[1], truth[2], truth[3]
                xmax = xmin + w
                ymax = ymin + h
                truth[2], truth[3] = xmax, ymax
                truths[num].append(truth)
        f5.close()
        
#print(truths)     
with open("comp4_det_test_car.txt") as f2:
    for line in f2:
        #print(line)
        line = line.strip()
        linelist = line.split(' ')
        num = re.findall(pattern, linelist[0])
        num  = num[0]
        #print('num', num)
        if num not in probs:
            probs[num] = []
        prob = linelist[1:6]
        #print ('prob', prob)
        for index, item in enumerate(prob):
            prob[index] = float(item)
        #print ('prob', prob)
        probs[num].append(prob)
with open("comp4_det_test_plane.txt") as f3:
    for line in f3:
        line = line.strip()
        linelist = line.split(' ')
        num = re.findall(pattern, linelist[0])
        num  = num[0]
        if num not in probs:
            probs[num] = []
        prob = linelist[1:6]
        for index, item in enumerate(prob):
            prob[index] = float(item)
        probs[num].append(prob)

proposals = 0
thresh = 0.25
iouthresh = 0.5
total = 0
avg_iou = 0
correct = 0
for id in idlist:
    #print(item)
    #print(probs[item]
    itruths = truths[id]
    iprobs = probs[id]
    for item in iprobs:
        if item[0] > thresh:
            proposals = proposals + 1
    for index, itruth in enumerate(itruths):
        total = total + 1
        best_iou = 0
        for iprob in iprobs:
            ibox = iprob[1:5]
            print
            iou = box_iou(ibox, itruth)
            if ((iprob[0] > thresh) and (iou > best_iou)):
                best_iou = iou
        avg_iou = avg_iou + best_iou
        if (best_iou > iouthresh):
            correct = correct + 1
precision = float(correct/proposals)
recall = float(correct/total)
print('precision:', precision)
print('recall', recall)
